# Imports

In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection

# Imports of other project files
from log_wrapper import LogWrapper
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex)

# Other imports
import importlib

import scipy.interpolate

In [ ]:
# Matplotlib configuring
plt.style.use(['science', 'no-latex'])
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Simulation setup
## Pre-finalize steps

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
v_stiction=1e-3
plant.set_stiction_tolerance(v_stiction)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
# def_joint_angles = list(np.zeros(num_links))
# def_joint_angles[0] = np.pi/100
# def_joint_angles[1] = np.pi/100
# def_joint_angles[int(num_links*0.65)-2] = -np.pi/10
# def_joint_angles[int(num_links*0.65)-1] = -np.pi/10
# def_joint_angles[int(num_links*0.65)] = -np.pi/10
# def_joint_angles[int(num_links*0.65)+1] = -np.pi/10
# # def_joint_angles[int(num_links*0.65)+2] = -np.pi/10
def_joint_angles = 0#np.pi/3

paper = Paper(plant, scene_graph, num_links, "NATURAL", default_joint_angle=def_joint_angles,
              stiffness=0,#2.5e-2,
              damping=0,)#7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

finger_instance, finger_body = finger.AddFinger(plant, constants.INIT_Y, constants.INIT_Z)

# Set up logger (needs to happen after all bodies are added)
log_wrapper = LogWrapper(plant.num_bodies(), int(finger_body.index()), paper)
builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
paper.plant.get_body(BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]

## Controller selection

In [ ]:
## CHOOSE CONTROL SYSTEM HERE BY UNCOMMENTING
# # PD control: hits too low
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )

# PD control: hits too high
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )


# Edge feedback
finger_ctrlr = finger.EdgeController(
    plant,
    paper,
    int(finger_instance),
    F_Nd=2,
    debug=True,
)

# # Optimization controller
# finger_ctrlr = finger.OptimizationController(
#     plant,
#     paper,
#     int(finger_instance),
#     paper.get_free_edge_instance()
# )

# Blank controller
# finger_ctrlr = finger.BlankController(
#     plant,
#     int(finger_instance)
# )

## Post-finalize steps

In [ ]:
plant.Finalize()

## Post finalize steps
# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))

# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 
builder.Connect(plant.get_contact_results_output_port(), log_wrapper.get_input_port(3))
builder.Connect(plant.get_reaction_forces_output_port(), log_wrapper.get_input_port(4))

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
if type(finger_ctrlr) is finger.OptimizationController:
    finger_ctrlr.optimize(plant.GetPositions(diagram_context))

# Run simulation

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)

# This way, we can use the actual time the simulation ran for
effective_tspan = max(logger.sample_times())

vis.stop_recording()
vis.publish_recording()

# Plots used in orginal paper

In [ ]:
# This script is getting huge. We need to turn off parts that we're not using.
generate_orig_plots = False

In [ ]:
if generate_orig_plots:
    # Plot manipulator position vs. trajectory
    # PROGRAMMING: Use body index for finger instead of model instance
    if type(finger_ctrlr) is finger.PDFinger:
        plt.figure(figsize=(2*3,2*2))
        plt.plot(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1],
                 logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2],
                 label='Manipulator position')
        plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, label='Trajectory')
        plt.xlabel("$y$ position")
        plt.ylabel("$z$ position")
        plt.legend()
        plt.show()

In [ ]:
if generate_orig_plots:
    # Generate plots for paper
    nb = plant.num_bodies()
    y_traces = []
    z_traces = []
    theta_traces = []
    for b in paper.link_idxs:
        y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
        z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
        theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
    y_traces = np.array(y_traces)
    z_traces = np.array(z_traces)
    theta_traces = np.array(theta_traces)

    times_ = np.arange(0,effective_tspan, effective_tspan/10)
    cmap = cm.get_cmap("viridis_r")
    plt.figure(figsize=(2*3,2*2))
    if type(finger_ctrlr) is finger.PDFinger:
        plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, '--k', zorder=-1)
    for t in times_:
        c = cmap(t/effective_tspan)
        idx = np.argmax(logger.sample_times() >= t)

        # Plot paper
        # PROGRAMMING: Account for paper thickness in plots
        for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
            y0 = y - np.cos(theta)*paper.link_width/2
            z0 = z - np.sin(theta)*paper.link_width/2
            y1 = y + np.cos(theta)*paper.link_width/2
            z1 = z + np.sin(theta)*paper.link_width/2
            plt.plot([y0, y1], [z0, z1], color=c)


        # Plot manipulator
        plt.scatter(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1,idx], 
                    logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2,idx],
                    color=c, s=300, zorder=1)

    xlim = plt.xlim()
    ylim = plt.ylim()
    plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
    plt.xlim(xlim)
    plt.ylim(ylim)
    cb = plt.colorbar()
    cb.set_label("Time")
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.show()

# Drag plots

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
generate_drag_plots = False

In [ ]:
if generate_drag_plots:
    # Plot drag forces
    plt.figure(figsize=(16, 12))
    nb = plant.num_bodies
    vel_traces = []
    for b in paper.link_idxs:
        rho = 1.225
        C_d = paper.link_width*paper.depth
        vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
        f = 0.5*rho*C_d*vel_sqred
        plt.plot(f)

    plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
    plt.show()

# Verifying equations hold
According to my linear algebra:
$$
\begin{aligned}
     m_La_{LT} &= F_{FL} + F_{GT} +F_{OT} \\
    m_La_{LN} &= F_{NL} + F_{GN} +F_{ON} \\
    m_Ma_{MT} &= F_{FM} + F_{CT} \\
    m_Ma_{MN} &= F_{NM} + F_{CN} \\
    a_{LT} &= a_{MT} \\
    a_{LN} &= a_{MN} \\
    a_{LT} &= -\frac{w_L}{2}\dot\theta^2 \\
    a_{LN} &= \frac{w_L}{2}\ddot\theta \\
    I_L\ddot\theta &= \frac{w_L}{2}F_{ON}- \frac{h_L}{2}F_{FL} - r_TF_{NL} \\
    F_{NL} &= -F_{NM} \\
    F_{FL} &= -F_{FM} \\
    a_{NL} &= a_{Nd}  \\
    a_{FL} &= 0
\end{aligned}
$$
For each of these equations, I want to plot the value on the left side and the value on the right side to see if they match.

## Loading signals
Preparing all the signals we'll need.

In [ ]:
debug = finger_ctrlr.debug
times = np.array(debug['times'])

### Inputs
These should all come from simulation/geometry, since they are inputs to the controller and are "fixed" from the controller's perspective.
#### Geometric quantities ($m_L, m_M, w_L, I_L, h_L, \mu$)

In [ ]:
m_L = paper.link_mass
m_M = constants.FINGER_MASS
w_L = paper.link_width
I_L = paper.plant.get_body(
    BodyIndex(paper.get_free_edge_idx())).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
h_L = h_L = paper.height
mu = constants.FRICTION

#### Vectors which aren't determined by the force balance ($F_{GT}, F_{GN} \hat T, \hat N$)
- [ ] PROGRAMMING: also get $\hat T$ from simulation somehow

In [ ]:
N_hats_raw =  logger.data()[log_wrapper.contact_entry_start_idx+8:log_wrapper.contact_entry_start_idx+11,:]
N_hats = scipy.interpolate.interp1d(
                     logger.sample_times(),    
                     np.expand_dims(N_hats_raw.T, 2), axis=0)(times)
N_hats.shape

In [ ]:
T_hats = np.array(debug['T_hats'])
T_hats.shape

It's also useful here to calculate project matrices:

In [ ]:
T_hat_proj_mat = np.matmul(np.array(debug['T_hats']), np.transpose(np.array(debug['T_hats']), [0, 2, 1]))

In [ ]:
N_hat_proj_mat = np.matmul(np.array(debug['N_hats']), np.transpose(np.array(debug['N_hats']), [0, 2, 1]))

- [ ] PROGRAMMING: Calc these directly instead
- [ ] PROGRAMMING: Stop interpolating everything (avoid using `debug`)

In [ ]:
F_GT = np.array(debug['F_GTs'])
F_GN = np.array(debug['F_GNs'])

#### Positions and velocities which are "fixed'' until the next timestep ($\dot\theta^2, r_T$)

In [ ]:
d_theta_no_interp = logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx() + 6 + 3]
d_theta = scipy.interpolate.interp1d(logger.sample_times(), d_theta_no_interp, axis=0)(times)

Note that $r_T$ here is calculated differently (but more accurately) than in the controller.

In [ ]:
p_contact_no_interp = np.expand_dims(
    logger.data()[log_wrapper.contact_entry_start_idx+5:log_wrapper.contact_entry_start_idx+8,:].T,
    2)
p_contact = scipy.interpolate.interp1d(logger.sample_times(), p_contact_no_interp, axis=0)(times)

In [ ]:
p_link_no_interp = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx():
                  log_wrapper.entries_per_body*paper.get_free_edge_idx()+3].T,
    2)
p_link = scipy.interpolate.interp1d(logger.sample_times(), p_link_no_interp, axis=0)(times)

In [ ]:
p_link_contact = p_contact - p_link
p_link_contact_T_vec = np.matmul(T_hat_proj_mat, p_link_contact)
p_link_contact_T_mag = np.linalg.norm(p_link_contact_T_vec, axis=1)
p_link_contact_T_sgn = np.sign(np.matmul(
    np.transpose(np.array(debug['T_hats']), [0, 2, 1]),
    p_link_contact_T_vec))
r_T = p_link_contact_T_mag.flatten()*p_link_contact_T_sgn.flatten()

#### Control inputs ($a_{Nd}$)
- [ ] PROGRAMMING: Pass in `a_Nd` instead of `F_Nd`

In [ ]:
a_Nd = finger_ctrlr.F_Nd/paper.link_mass

### Outputs
These should all come directly from simulation, except for control forces.
#### Accelerations of both objects ($a_{LT}, a_{LN}, a_{MT}, a_{MN}, \ddot\theta$)

In [ ]:
a_link_raw = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12:
                     log_wrapper.entries_per_body*paper.link_idxs[-1]+12+3,:]
a_link_no_interp = np.expand_dims(a_link_raw.T, 2)
a_link = scipy.interpolate.interp1d(logger.sample_times(),
                                    a_link_no_interp,
                                    axis=0)(times)

In [ ]:
a_LT_vec = np.matmul(T_hat_proj_mat, a_link)
a_LT_mag = np.linalg.norm(a_LT_vec, axis=1)
a_LT_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    a_LT_vec))
a_LT = a_LT_mag.flatten()*a_LT_sgn.flatten()

In [ ]:
a_LN_vec = np.matmul(N_hat_proj_mat, a_link)
a_LN_mag = np.linalg.norm(a_LN_vec, axis=1)
a_LN_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    a_LN_vec))
a_LN = a_LN_mag.flatten()*a_LN_sgn.flatten()

In [ ]:
a_man_raw = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 12:
                     log_wrapper.entries_per_body*int(finger_body.index())+12+3,:]
a_man_no_interp = np.expand_dims(a_man_raw.T, 2)
a_man = scipy.interpolate.interp1d(logger.sample_times(),
                                    a_man_no_interp,
                                    axis=0)(times)

In [ ]:
a_MT_vec = np.matmul(T_hat_proj_mat, a_man)
a_MT_mag = np.linalg.norm(a_MT_vec, axis=1)
a_MT_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    a_MT_vec))
a_MT = a_MT_mag.flatten()*a_MT_sgn.flatten()

In [ ]:
a_MN_vec = np.matmul(N_hat_proj_mat, a_man)
a_MN_mag = np.linalg.norm(a_MN_vec, axis=1)
a_MN_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    a_MN_vec))
a_MN = a_MN_mag.flatten()*a_MN_sgn.flatten()

In [ ]:
dd_theta_no_interp = logger.data()[log_wrapper.entries_per_body*paper.get_free_edge_idx() + 12 + 3,:]
dd_theta = scipy.interpolate.interp1d(logger.sample_times(), dd_theta_no_interp, axis=0)(times)

#### Contact, object, and control forces on each object ($F_{NL}, F_{FL}, F_{ON}, F_{OT}, F_{NM}, F_{FM}, F_{CT}, F_{CN}$

##### Contact

In [ ]:
F_contact_M_raw = logger.data()[log_wrapper.contact_entry_start_idx+0:log_wrapper.contact_entry_start_idx+3,:]
F_contact_M = scipy.interpolate.interp1d(logger.sample_times(), 
                                         np.expand_dims(F_contact_M_raw.T, 2), axis=0)(times)

In [ ]:
F_NM_vec = np.matmul(N_hat_proj_mat, F_contact_M)
F_FM_vec = F_contact_M - F_NM_vec

In [ ]:
F_NM_mag = np.linalg.norm(F_NM_vec, axis=1)
F_NM_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    F_NM_vec))
F_NM = F_NM_mag.flatten()*F_NM_sgn.flatten()

In [ ]:
F_FM_mag = np.linalg.norm(F_FM_vec, axis=1)
F_FM_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    F_FM_vec))
F_FM = F_FM_mag.flatten()*F_FM_sgn.flatten()

In [ ]:
F_FL = -F_FM
F_NL = -F_NM

##### Object

In [ ]:
joint_force_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx:log_wrapper.joint_entry_start_idx+3,:]

In [ ]:
F_OT = scipy.interpolate.interp1d(logger.sample_times(), joint_force_in_compliance_frame[1,:], axis=0)(times)
F_ON = scipy.interpolate.interp1d(logger.sample_times(), joint_force_in_compliance_frame[2,:], axis=0)(times)

##### Control

In [ ]:
F_CT = np.array(debug['F_CTs'])
F_CN = np.array(debug['F_CNs'])

## Plotting settings

In [ ]:
lw = 5

## Plots

### $m_La_{LT} = F_{FL} + F_{GT} +F_{OT}$

In [ ]:
lhs_label = r'$m_La_{LT}$'
rhs_label = r'$F_{FL} + F_{GT} +F_{OT}$'

In [ ]:
lhs = m_L*a_LT

In [ ]:
rhs = F_FL+F_GT+F_OT

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-20, 5)
plt.legend()
plt.xlabel("Time")
plt.show()

### $m_La_{LN} = F_{NL} + F_{GN} +F_{ON}$

In [ ]:
lhs_label = r'$m_La_{LN}$'
rhs_label = r'$F_{NL} + F_{GN} +F_{ON}$'

In [ ]:
lhs = m_L*a_LN

In [ ]:
rhs = F_NL + F_GN + F_ON

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-2, 5)
plt.legend()
plt.xlabel("Time")
plt.show()

### $m_Ma_{MT} = F_{FM} + F_{CT}$

In [ ]:
lhs_label = r'$m_Ma_{MT}$'
rhs_label = r'$F_{FM} + F_{CT}$'

In [ ]:
lhs = m_M*a_MT

In [ ]:
rhs = F_FM + F_CT

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=0.5)
plt.ylim(-0.5, 0.05)
plt.legend()
plt.xlabel("Time")
plt.show()

### $m_Ma_{MN} = F_{NM} + F_{CN}$

In [ ]:
lhs_label = r'$m_Ma_{MN}$'
rhs_label = r'$ F_{NM} + F_{CN}$'

In [ ]:
lhs = m_M*a_MN

In [ ]:
rhs = F_NM+F_CN

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-0.5, 0.5)
plt.legend()
plt.xlabel("Time")
plt.show()

### $a_{LT} = a_{MT}$

In [ ]:
lhs_label = r'$a_{LT}$'
rhs_label = r'$a_{MT}$'

In [ ]:
lhs = a_LT

In [ ]:
rhs = a_MT

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=0.5)
plt.ylim(-150, 10)
plt.legend()
plt.xlabel("Time")
plt.show()

### $a_{LN} = a_{MN}$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$a_{MN}$'

In [ ]:
lhs = a_LN

In [ ]:
rhs = a_MN

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-10, 40)
plt.legend()
plt.xlabel("Time")
plt.show()

#### Supporting plot

In [ ]:
plt.figure(figsize=(16, 16))
lw = 5
plt.plot(times, a_link[:, 0,:], label='$a_{Lx}$', color='pink', linewidth=lw)
plt.plot(times, a_link[:, 1,:], label='$a_{Ly}$', color='lightgreen', linewidth=lw)
plt.plot(times, a_link[:, 2,:], label='$a_{Lz}$', color='lightskyblue', linewidth=lw)
plt.plot(times, a_man[:,0,:], '--', label='$a_{Mx}$', color='red', linewidth=lw, alpha=0.5)
plt.plot(times, a_man[:,1,:], '--', label='$a_{My}$', color='green', linewidth=lw, alpha=0.5)
plt.plot(times, a_man[:,2,:], '--', label='$a_{Mz}$', color='blue', linewidth=lw, alpha=0.5)
plt.xlabel("Time (seconds)")
plt.ylim(-100, 100)
plt.legend()
plt.show()

### $a_{LT} = -\frac{w_L}{2}\dot\theta^2$

In [ ]:
lhs_label = r'$a_{LT}$'
rhs_label = r'$-\frac{w_L}{2}\dot\theta^2$'

In [ ]:
lhs = a_LT

In [ ]:
rhs = -(w_L/2)*d_theta**2

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-150, 2)
plt.legend()
plt.xlabel("Time")
plt.show()

### $a_{LN} = \frac{w_L}{2}\ddot\theta$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$\frac{w_L}{2}\ddot\theta$'

In [ ]:
lhs = a_LN

In [ ]:
rhs = dd_theta*w_L/2

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)

plt.ylim(-10, 30)
plt.legend()
plt.xlabel("Time")
plt.show()

### $I_L\ddot\theta = -\frac{w_L}{2}F_{ON}+ \frac{h_L}{2}F_{FL} + r_TF_{NL}$

In [ ]:
lhs_label = r'$I_L\ddot\theta$'
rhs_label = r'$-\frac{w_L}{2}F_{ON}+ \frac{h_L}{2}F_{FL} + r_TF_{NL}$'

In [ ]:
lhs = I_L*dd_theta

In [ ]:
rhs = -F_ON*w_L/2+h_L*F_FL/2+r_T*F_NL

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-0.3, 0.3)
plt.legend()
plt.xlabel("Time")
plt.show()

#### Supporting/debugging plots

In [ ]:
lw = 5
plt.figure(figsize=(16, 16))
plt.plot(times, r_T, label="Simulation", linewidth=5, color='lightskyblue', zorder=0)
plt.plot(times, paper.link_width + np.array(debug['d_Ts']) - paper.link_width/2, 
         label="Controller", linewidth=5, color='b', linestyle='--', zorder=1)
plt.show()

### $F_{NL} = -F_{NM}$

In [ ]:
lhs_label = r'$F_{NL}$'
rhs_label = r'$-F_{NM}$'

In [ ]:
lhs = F_NL

In [ ]:
rhs = -F_NM

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(0, 4)
plt.legend()
plt.xlabel("Time")
plt.show()

### $F_{FL} = -F_{FM}$

In [ ]:
lhs_label = r'$F_{FL}$'
rhs_label = r'$-F_{FM}$'

In [ ]:
lhs = F_FL

In [ ]:
rhs = -F_FM

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-0.5, 1)
plt.legend()
plt.xlabel("Time")
plt.show()

#### Supporting plot

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, abs(F_FL), label=r"$\left|F_{FL}\right|$", linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(times, abs(F_FM), label=r"$\left|F_{ML}\right|$", linewidth=lw, color='b', linestyle='--', zorder=1)
plt.plot(times, abs(F_NL)*constants.FRICTION, label=r"$\mu\left|F_{NL}\right|$", linewidth=lw, color='k', linestyle=':')
plt.ylim(-0.5, 2)
plt.legend()
plt.xlabel("Time")
plt.show()

### $a_{LN} = a_{Nd}$

In [ ]:
lhs_label = r'$a_{LN}$'
rhs_label = r'$a_{Nd}$'

In [ ]:
lhs = a_LN

In [ ]:
rhs = a_Nd

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.axhline(rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-10, 50)
plt.legend()
plt.xlabel("Time")
plt.show()

### $F_{FL} = 0$

In [ ]:
lhs_label = r'$F_{FL}$'
rhs_label = r'0'

In [ ]:
lhs = F_FL

In [ ]:
rhs = 0

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, lhs, label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0)
plt.axhline(rhs, label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1)
plt.ylim(-0.5, 2)
plt.legend()
plt.xlabel("Time")
plt.show()

# Other useful plots

In [ ]:
# Get debug dict
debug = finger_ctrlr.debug
times = np.array(debug['times']) # np.arange(len(debug['d_Ns']))*effective_tspan/len(debug['d_Ns'])
dt = effective_tspan/len(debug['d_Ns'])

## Verifying we have the same $\hat N$

In [ ]:
N_hats_sim = N_hats

In [ ]:
N_hats_ctrl = np.array(debug['N_hats'])

In [ ]:
plt.figure(figsize=(16, 8))
lw = 5
plt.plot(times, N_hats_sim[:,0,:], label='$\hat n_{BA_W,x}$', color='pink', linewidth=lw)
plt.plot(times, N_hats_sim[:,1,:], label='$\hat n_{BA_W,y}$', color='lightgreen', linewidth=lw)
plt.plot(times, N_hats_sim[:,2,:], label='$\hat n_{BA_W,z}$', color='lightskyblue', linewidth=lw)
plt.plot(times, N_hats_ctrl[:,0,:], '--', label='$\hat N_x$', color='red', linewidth=lw)
plt.plot(times, N_hats_ctrl[:,1,:], '--', label='$\hat N_y$', color='green', linewidth=lw)
plt.plot(times, N_hats_ctrl[:,2,:], '--', label='$\hat N_z$', color='blue', linewidth=lw)
plt.xlabel("Time (seconds)")
plt.legend()
plt.show()

## $d_T$, $d_N$, separation speed, and slip speed

In [ ]:
separation_speed = logger.data()[log_wrapper.contact_entry_start_idx+3,:]
slip_speed = logger.data()[log_wrapper.contact_entry_start_idx+4,:]

In [ ]:
lw=5

plt.figure(figsize=(16, 8))
plt.plot(times, np.array(debug['d_Ns']), label='$d_N$', linewidth=lw)
plt.plot(times, debug['d_Ts'], label='$d_T$', linewidth=lw)
plt.axhline(-(constants.FINGER_RADIUS), label='Contact $d_N$ (finger radius)', linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(logger.sample_times(), separation_speed, label="Separation speed", linewidth=lw)
plt.axhline(0, color='k', linestyle='--', label='Contact', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Velocity (m/s)")
plt.show()

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(logger.sample_times(), slip_speed, label="slip speed", linewidth=5)
plt.axhline(v_stiction, linestyle="--", color="k", label="$v_{stiction}$", linewidth=5)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
plt.ylim(0, 2*v_stiction)
plt.show()

### Contact point
 - [ ] PROGRAMMING: Add arrow heads to contact point plots

In [ ]:
contact_point = logger.data()[log_wrapper.contact_entry_start_idx+5:log_wrapper.contact_entry_start_idx+8,:]

In [ ]:
plt.figure(figsize=(12,12))

plt.plot(contact_point[1,:], contact_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()

In [ ]:
man_point = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
                          log_wrapper.entries_per_body*int(finger_body.index())+3,:]

In [ ]:
plt.figure(figsize=(12,12))
plt.gca().add_patch(plt.Circle((0, 0), constants.FINGER_RADIUS, color='r'))
plt.plot(contact_point[1,:]-man_point[1,:], contact_point[2,:]-man_point[2,:], color='k', linestyle='--', linewidth='5')
plt.gca().set_aspect('equal', 'box')
plt.show()